In [1]:
%matplotlib
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import time
from opt import nlp_solve, reset_x0

Using matplotlib backend: <object object at 0x7fa7947195b0>


In [2]:
class Gaussian(object):
    def __init__(self, mean, cov):
        self._mean = mean
        self._cov = cov
        self._dim = self._mean.shape[-1]
        self._k = 1/( np.power(2*np.pi, self._dim/2)* np.sqrt(np.linalg.norm(self._cov)) )
    
    def reset(self, mean, cov):
        self._mean = mean
        self._cov = cov
        self._dim = self._mean.shape[-1]
        self._k  = 1/( np.power(2*np.pi, self._dim/2)*np.sqrt(np.linalg.norm(self._cov)) )
    
    def __call__(self, x):
        e = x-self._mean
        
        ee = e*(np.linalg.inv(self._cov)@e)
        ee = np.sum(ee, axis=0)
        return self._k*np.exp(-ee/2)

In [3]:
class DynObstacle():
    def __init__(self, trj:np.array, dt=0.1):
        self._dt = dt
        self._trj = trj
        self._total_steps = trj.shape[-1]
        
        self._step = 0
        self.pos = trj[:,0]
        self.pos_cov = np.array([[0.1, 0],
                                  [0, 0.1]])
        self.distribution = Gaussian(self.pos.reshape([-1,1]), self.pos_cov)

    def get_pred(self):
        pred = np.zeros(6*10)

        pred[0:2] = self.pos
        pred[2:6] = self.pos_cov.T.reshape(4)
        tt = self._step
        for i in range(1,10):
            pred[0+6*i:2+6*i] = self._trj[:, tt]
            pred[2+6*i:6+6*i] = self.pos_cov.T.reshape(4)
            tt += 1
            if tt >= self._total_steps:
                tt = 0
        return pred

    def update(self):
        self.pos = self._trj[:, self._step]
        self.distribution.reset(self.pos.reshape([-1, 1]), self.pos_cov)
        
        self._step += 1
        if self._step >= self._total_steps:
            self._step = 0

In [4]:
class ProbabilityMap(object):
    def __init__(self, size=(5,5)):
        self._size = size
        self._obstacles = []

        self.grid_map = np.zeros([100, 100])

        x = np.linspace(0, self._size[0], num=100)
        y = np.linspace(0, self._size[1], num=100)
        X, Y = np.meshgrid(x, y)
        X = X.reshape([1, -1])
        Y = Y.reshape([1, -1])
        self._XY = np.concatenate([X, Y], axis=0)
    
    def add_obstacle(self, obs):
        self._obstacles.append(obs)

    def get_pred(self):
        return self._obstacles[0].get_pred()
            
    def map_update(self):
        self.grid_map = np.zeros([100, 100])
        for obs in self._obstacles:
            obs.update()
            self.grid_map += obs.distribution(self._XY).reshape([100, 100])
        

In [5]:
map = ProbabilityMap()
mean = np.array([2.5,2.5]).reshape([-1,1])
cov = np.array([[0.4, 0.3], 
                [0.1, 1]])

t = np.linspace(0, 2*np.pi, 50)
x = np.sin(t)*2 + 2.5
y = np.ones(x.shape[0])*2
x = x.reshape([1, -1])
y = y.reshape([1, -1])
trj = np.concatenate([x,y], axis=0)
obs = DynObstacle(trj)
map.add_obstacle(obs)

In [7]:

sim_dt = 0.1
def sim_run(T=5):
    global map
    t = 0

    x_init = np.array([2,0,0,0])
    x_goal = np.array([2.5, 4])
    p = np.zeros(4+6*10+2)
    pos = np.zeros([2,10])
    
    reset_x0()

    while t<T:
        map.map_update()
        
        p[:4] = x_init
        p[4:4+6*10] = map.get_pred()
        p[4+6*10:] = x_goal
        
        t1 = time.time()
        nlp_res = nlp_solve(p)
        t2 = time.time()
        print(t2-t1)
        X = np.array(nlp_res['x']).reshape(-1)
        x_init = X[2:6]

        for i in range(10):
            pos[0,i] = X[2+i*6]
            pos[1,i] = X[3+i*6]

        t += sim_dt
        yield map, pos

sim = sim_run()

fig = plt.figure()
ax = fig.add_axes([0.03, 0.05, 0.94, 0.9], animated=True)
im = ax.imshow(map.grid_map, origin='lower', extent=(0,5, 0,5), animated=True)
trj, = ax.plot([],[], animated=True)

def update_plot(*args):
    map, pos = next(sim)
    im.set_array(map.grid_map)
    trj.set_data(pos[0,:], pos[1,:])
    # print(pos)
    return im,trj

ani = animation.FuncAnimation(fig, update_plot, interval=100, blit=True, repeat=False)
plt.show(block=True)

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 386.00us ( 42.89us) 113.64us ( 12.63us)         9
       nlp_g  | 421.00us ( 46.78us) 124.96us ( 13.88us)         9
    nlp_grad  | 119.00us (119.00us)  25.02us ( 25.02us)         1
  nlp_grad_f  | 500.00us ( 50.00us) 124.59us ( 12.46us)        10
  nlp_hess_l  | 554.00us ( 69.25us) 118.48us ( 14.81us)         8
   nlp_jac_g  | 126.00us ( 12.60us)  62.47us (  6.25us)        10
       total  |  68.09ms ( 68.09ms)  17.15ms ( 17.15ms)         1
0.021010398864746094
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 580.00us ( 72.50us) 163.36us ( 20.42us)         8
       nlp_g  | 942.00us (117.75us) 203.83us ( 25.48us)         8
    nlp_grad  |  88.00us ( 88.00us)  25.95us ( 25.95us)         1
  nlp_grad_f  | 686.00us ( 76.22us) 145.54us ( 16.17us)         9
  nlp_hess_l  | 549.00us ( 78.43us) 137.69us ( 19.67us)         7
   nlp_jac_g  | 426.00us ( 47.33us)  80.05us (  8.89us)